# Fixing a Pin Cell problem

## Scenario
* Given a model `models/pin_cell.imcnp` from dubious origins
    * Meant to represent a W17$\times$17 pin cell
* However, has a few Problems that need to be fixed first


## First, let's just print out the file.

In [ ]:
!cat models/pin_cell.imcnp

# Plot

---
**NOTE**

You will need MCNP to plot geometries. MCNP is distributed through The [Radiation Safety Information Computational Center (RSICC)](https://rsicc.ornl.gov/) at Oak Ridge National Laboratory and cannot be provided in this workshop. The depicted plots are for information only.

---

![pin cell plot](figs/pin_cell.png)

# Improvement: Goals
* Add helium to gas gap
* Fix Zircaloy
   * Elemental cross sections are generally the wrong tool
   * Mixing different nuclear data libraries is a bad idea
* Fix (lack of) reflective boundaries
* Set this up for running Kcode

# Step 1: Import Python modules

In [ ]:
# actually needed
import montepy
import numpy as np

# for pretty Jupyter web pages Usually not needed
from _config import IFrame

# Step 2: Open the File in MontePy
1. Read the docs

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.input_parser.input_reader.html#montepy.input_parser.input_reader.read_input",
)

2. This function does appear to be the correct one.
     * destination is required
     * `mcnp_version` is optional. We won't be using specific features that require changing this
     * replace is optional. Here the default seems like a safe bet.
### tip: this function is available at the top level: `montepy.read_input`

In [ ]:
problem = montepy.read_input("models/pin_cell.imcnp")

# Step 3: Explore the Problem

## Goals
* print the following
   * the cells, surfaces, and materials in the problem
   * the version of MCNP this is designed for

In [ ]:
IFrame("https://www.montepy.org/en/stable/api/montepy.mcnp_problem.html")

In [ ]:
print(problem.cells)
print(problem.surfaces)
print(problem.materials)
print("version", problem.mcnp_version)

# Goal 1: Fix Boundary Conditions
## Reminder
* Reflective boundaries are a property of the surface; not the cell
## Things to figure out:
1. How to get the specific surfaces:
   * Did you notice that planes (e.g., `PZ`) were only used as boundaries?
2. How to set a reflective boundary

# Task 1.1 Is there an easy way to get a surface by type?

In [ ]:
IFrame("https://www.montepy.org/en/stable/api/montepy.surface_collection.html")

## Let's explore these surface type generators.
* Hint: generators are a specific type of iterator
### Task: Print all surfaces of type `PZ`

In [ ]:
# Task: Print every "PZ" type of surface
z_surfaces = None  # Change this line
for surface in z_surfaces:
    print(repr(surface))

# Task 1.2: Set Reflective Boundaries
* This is a property of surfaces
* Let's check those docs

## Task set one surface to be reflective
* Reminder: `surface` will be still set from the previous for loop, and should be a `PZ` surface

In [ ]:
IFrame("https://www.montepy.org/en/stable/api/montepy.surfaces.surface.html")

In [ ]:
surface.is_reflecting = True
print(surface.is_reflecting)

# Goal 1: Conclusion: Bring it all together
* Iterate over all `PX`, `PY`, and `PZ` surfaces
* Set all such surfaces to `is_reflecting`

* For the adventerous: check out [`itertools.chain`](https://docs.python.org/3/library/itertools.html#itertools.chain)

In [ ]:
# Task: Set all PX, PY, and PZ surfaces to reflecting
import itertools

iter_surfaces = itertools.chain()
for surface in iter_surfaces:
    surface.is_reflecting = True
    print(surface.mcnp_str())

# Goal 2: Fill the Gas Gap with Helium

## Tasks
1. Find the gas gap cell
2. Create a new helium cell
3. Assign that material to the gas gap cell
4. Set the density of the gas gap


# Task 2.1: Finding the Cell

Ways to find a cell:

1. Knowing its cell number _a priori_
    * Boring and error-prone
2. By its comments, if they are helpful
3. By its material
    * This is the only void cell
4. By its surfaces
    * This should be sandwiched between the smallest, and next smallest cylinder

In [ ]:
# We'll get a cell to play with
# this grabs the first cell
cell = list(problem.cells)[0]

# Explore a cell
## Task, print all of the following for the cell
* its number
* its comment(s)
* its surfaces
* its material

In [ ]:
IFrame("https://www.montepy.org/en/stable/api/montepy.cell.html")

In [ ]:
# Task: print the cell: number, comments, material, and surfaces
print()

# Task 2.1: Find the cell which is Void (material is `None`)

In [ ]:
# Task: set 'gap' to the void Cell
gap = None
for cell in problem.cells:
    # Find the void
    pass
if gap is not None:
    print(gap, gap.comments)

# Task 2.2 Define a Helium Material

Need to:
* Create a new material
* Add it to the problem

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.data_inputs.material.html",
)

In [ ]:
# Task: define 'helium' Material and add it to the problem
helium = None
if helium is not None and helium not in problem.materials:
    helium.number = problem.materials.request_number()
    problem.materials.append(helium)

# Adding Nuclide

Tips:
* Natural helium is mostly `He-4`
* It is good practice to specify the xslib.
  * Pick a library, say, ENDF/B-VII.1
  * Pick a temperature, say, 600 K
  * Refer to the ACE manual to find the suffix [LA-UR-17-20709](https://doi.org/10.2172/1342828)

In [ ]:
?montepy.Material.add_nuclide

In [ ]:
# Task: Add one or more helium isotopes to the 'helium'
# Hint: the ENDF/B-VII.1 600K continuous-energy library is "81c"
nuc = None
if nuc is not None:
    helium.add_nuclide(nuc, 1.0)
    print(helium.is_atom_fraction)

# Task 2.3: Finding Density
* PWR helium gas gap pressure starts at around 25 atm (according to [nuclear-power.com](https://www.nuclear-power.com/nuclear-power-plant/nuclear-fuel/fuel-assembly/fuel-rods-fuel-pins/))
* From [Ideal gas law](https://en.wikipedia.org/wiki/Ideal_gas_law)
 $$ p = \frac{nRT}{N_A}$$
* With:
    * $p$ being the pressure. 
    * $n$ being the atomic density
    * $R$ is the ideal gas constant. You can use:$8.2057\times10^{-5}\rm\frac{m^3atm}{K\cdot mol}$
    * $N_A$ is Avogadro's constant. You can use: $ 6.022\times 10^{23}$
    * $T$ is the temperature. Let's assume 600 K.
* Solve for the atom density, and convert it to units of $\rm\frac{a}{barn\cdot cm}$ (1 barn = $1\times10^{-24}\rm cm^2$)

 $$ p = \frac{nRT}{N_A}$$

In [ ]:
R = 8.2057e-5  # m3*atm/K-mol
N_A = 6.022e23
BARNS_TO_CM2 = 1e24
PRESSURE = 25  # atm
TEMP = 600  # K

In [ ]:
# Task: calculate the density
density_m3 = 0
density_cm3 = density_m3 / (100**3)  # (cm -> m)^3
density = density_cm3 / BARNS_TO_CM2
print(f"{density:.3e} at/b-cm")

# Task 2.4: Bring it all together
* Assign cell material
* Assign density

In [ ]:
IFrame("https://www.montepy.org/en/stable/api/montepy.cell.html")

In [ ]:
# Density is stored in the cells in MCNP
# Task: Set the material and density of the gap cell
gap

# Goal 3: Fix the Zircaloy Material definition
* Find the zircaloy
* Remove elemental nuclides
* Replace with proper isotopic break downs

## Task 3.1 find Zircaloy
* Easy to find as only material with Zr
* `Materials` have some helpful functions for this

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.materials.html#montepy.materials.Materials.get_containing_any",
)

In [ ]:
# Task: Set 'zirc' to the material containing Zirconium (Atomic symbol: Zr; Z: 40)
zirc = None
if zirc is not None:
    print(repr(zirc))

# Task 3.2 Define new components

* Take existing components
* Split into isotopic abundances and multiply with components
* Remove old components

In [ ]:
NAT_ABUNDANCES = {
    "Zr": {90: 0.5145, 91: 0.1122, 92: 0.1715, 94: 0.1738, 96: 0.0280},
    "Sn": {112: 0.0097, 114: 0.0066, 116: 0.0034, 117: 0.0768},
    "Fe": {54: 0.05845, 56: 0.91754, 57: 0.02119, 58: 0.00282},
    "Cr": {50: 0.04345, 52: 0.83789, 53: 0.09501, 54: 0.02365},
}
# From Meija, et al. "Isotopic Compositions of the Elements (IUPAC Technical Report) <https://doi.org/10.1515/pac-2015-0503>
# given in atom fraction.

# Play around with Materials
* Review [Material documentation](https://www.montepy.org/en/stable/api/montepy.data_inputs.material.html)
* Adding to a list while you are iterating over it is dangerous; make sure to:
    1. Create a new list of the components
    2. Clear the old material
    3. Add the new components to the material.
    4. Change libraries for all nuclides all at once.

In [ ]:
# Task: Replace all components of material 'zirc'
zirc2 = zirc.clone()
zirc2.clear()
for nuclide, base_fraction in zirc:
    element = nuclide.element
    abundances = NAT_ABUNDANCES[element.symbol]
    for A, iso_fraction in abundances.items():
        print(f"A: {A:3g} | fraction: {iso_fraction:}")
        # Now, add it to our new zirc2:
        print(f"{element.symbol}-{A}")
zirc2.change_libraries("82c")
zirc2

In [ ]:
# switch all cells over to new material
for cell in zirc.cells:
    cell.material = zirc2
# delete old material from problem
problem.materials.remove(zirc)

# Task 4 Set Up Eigenvalue run

* Need to define [ksrc](https://mcnp.lanl.gov/pdf_files/TechReport_2022_LANL_LA-UR-22-30006Rev.1_KuleszaAdamsEtAl.pdf#subsection.5.8.11), and [kcode](https://mcnp.lanl.gov/pdf_files/TechReport_2022_LANL_LA-UR-22-30006Rev.1_KuleszaAdamsEtAl.pdf#subsection.5.8.10).
* MontePy doesn't have a `KCODE` object yet. For arbitrary data inputs, MontePy can be given an arbitrary input string to add to the model.

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.mcnp_problem.html#montepy.mcnp_problem.MCNP_Problem.parse",
)

# Task 4.1
* Set one source site at the origin
* Set:
    * 100,000 histories per cycle
    * initial guess of k to be 1.1
    * Do 100 cycles
    * with 20 inactive cycles.

In [ ]:
# Task: Add kcode and ksrc to the problem
nparticles = int(1e5)
kguess = None
inactive_batches = None
total_batches = None
kcode = problem.parse(
    f"kcode {nparticles} {kguess} {inactive_batches} {total_batches}", append=False
)
# Now add a ksrc
ksrc = None

In [ ]:
# If 'append=False' earlier, append to the data block now.
problem.data_inputs.append(kcode)
problem.data_inputs.append(ksrc)

# Task 5 update water Density
* Moderator density is around standard temperature and pressure (roughly)
* Much too dense for a PWR
* Density should be 0.74 $\rm\frac{g}{cm^3}$ <sup>1</sup>

<sup>1</sup> N. E. Horelik et al., "Benchmark for Evaluation and Validation of Reactor Simulations (BEAVRS)," presented at the Int. Conf. Mathematics and Computational Methods Applied to Nuc. Sci. & Eng., Sun Valley, Idaho, 2013.

In [ ]:
WATER_DENSITY = 0.74  # [g/cm3]

# Task 5.1 Find moderator cell
* will find water material, then find the cell filled with that

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.materials.html#montepy.materials.Materials.get_containing_all",
)

In [ ]:
# get water
water = list(problem.materials.get_containing_all("H", "O", threshold=0.01))[0]
water

# Grab Cell

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.data_inputs.material.html#montepy.data_inputs.material.Material.cells",
)

In [ ]:
water_cell = list(water.cells)[0]
water_cell

# Set Cell Mass Density

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.cell.html#montepy.cell.Cell.mass_density",
)

In [ ]:
water_cell.mass_density = WATER_DENSITY

# Conclusion
* Write it out to a file `models/pin_cell_corrected.imcnp`

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.mcnp_problem.html#montepy.mcnp_problem.MCNP_Problem.write_problem",
)

In [ ]:
problem.write_problem("models/pin_cell_corrected.imcnp")

# Questions?